In [ ]:
import pandas as pd
import numpy as np
import re
import os
import json
from openpyxl import load_workbook

In [ ]:
# Load all the necessary Excel files to notebook

#file for contractor group
first_df = pd.read_excel('Copy of reconn year to date.xlsx',sheet_name ='con_ed_sli_-_field_inspection_r')

address_df = pd.read_excel('recon_addresses.xlsx',sheet_name='Sheet1') # addresses for contractor groups

#dictionary for address parsing (street names and arteries)
street_mapping_df = pd.read_excel('Copy of Dictionary master.xlsx', sheet_name='Street Names') 
artery_mapping_df = pd.read_excel('Copy of Dictionary master.xlsx', sheet_name='Art')

#file with default values and data standardization 
mapping_df = pd.read_excel('mapping2.xlsx',sheet_name = 'RECONN')

#blank template to convert to 
new_df = pd.read_excel('SLI_Template.xlsx')

with open('value_mapping.json', 'r') as f:
    value_mapping = json.load(f)

In [ ]:
file_to_read = input("Which database are you trying to run? \n ---> RECONN \n ---> SLI \n ---> STORTI \n")

if file_to_read == "RECONN":
    first_df[['corrosion_found_on_select_all_that_apply2', 'corrosion_location_select_all_that_apply','is_coned_piping_accessible','not_accessible_reason2','poesleeve_location','inspected_all_services_entering_building','any_impropersubstandard_conditions','corrosion_severity','corrosion_location_select_all_that_apply','do_you_have_access_to_the_gas_service_point_of_entry_poe','no_access_reason','is_the_gas_poe_sealed','how_did_you_seal_it']].replace('0','',inplace=True)
    first_df.iloc[:,1:] = first_df.iloc[:,1:].applymap(lambda x:x.upper() if isinstance(x,str) else x)
    first_df["meter_location_atmosphere_select_all_that_apply"] = first_df["meter_location_atmosphere_select_all_that_apply"].str.split(';').str[0]
elif file_to_read == 'SLI':
    # first_df[['CorrosionFoundOnCdCustomer', 'CorrosionLocationsCdCustomer','ImproperConditionsFoundCustomer','AccessibleConEd','NoAccessReasonCdConed','POESleeveLocationCd','InspDoneOnAllSvcsEnteringBldg','ImproperConditionsFoundConEd','CorrosionSeverityCdConEd','CorrosionFoundOnConEdRemarks','CorrosionLocationsCdConEd','PoeAccessible','PoeNoAccessReasonCd','PoeFoundSealedCondition','PoeSealTypeCd']].replace('0','',inplace=True)    
    first_df[['Corrosion found on?', 'Corrosion location? ','Any improper or substandard conditions?','Was the Con Edison pipe/equipment accessible?','No Access Reason2','Point of Entry (P.O.E) sleeve location?','Inspection Done on all Services Entering the Bldg?','Any Improper or Substandard Conditions?2','Corrosion severity on Con Ed piping?','Corrosion location? 2','Did you have access to Gas Service POE (point of entry)?','Reason Not Accessible','Gas Service Point of Entry Found Sealed?','How did you seal it?']].replace('0','',inplace=True)    
    first_df.iloc[:,1:] = first_df.iloc[:,1:].applymap(lambda x:x.upper() if isinstance(x,str) else x)    
    first_df["Meter Location Atmosphere "] = first_df["Meter Location Atmosphere "].str.split(',').str[0]
elif file_to_read == "STORTI":
    print("Incomplete")
else:
    raise ValueError("Invalid input, type exactly as options state")

# create a dictionary from the mapping_df file
mapping_dict = mapping_df.set_index('normalized name')['column name'].to_dict()
default_values_dict = mapping_df.set_index('normalized name')['default value'].apply(lambda x: x.split(',') if isinstance(x, str) else x).to_dict()
new_df = pd.DataFrame(columns = new_df.columns)

In [ ]:
for column in new_df.columns:
    if column in mapping_dict and mapping_dict[column] in first_df.columns:
        column_name = mapping_dict[column]
        new_df[column] = first_df[column_name]
    else:
        new_df[column]=new_df[column]

In [ ]:
# where value_mapping should be

In [ ]:
OutsideSet_map = value_mapping.get('OutsideSet',{})
PressureRegulatorInBuilding_map = value_mapping.get('PressureRegulatorInBuilding',{})
TypeOfDwellingCd_map = value_mapping.get('TypeOfDwellingCd',{})
MeterLocation_map = value_mapping.get('MeterLocation',{})
MeterLocationAtmosphere_map = value_mapping.get('MeterLocationAtmosphere',{})
PipingConstructionMaterialCd_map = value_mapping.get('PipingConstructionMaterialCd',{})
MeterConstructionMaterialCd_map = value_mapping.get('MeterConstructionMaterialCd',{})
FittingsConstructionMaterialCd_map = value_mapping.get('FittingsConstructionMaterialCd',{})
MainAndBranchService_map = value_mapping.get('MainAndBranchService',{})
NoAccessReasonCdBuilding_map = value_mapping.get('NoAccessReasonCdBuilding',{})
ServiceUnderStoop_map = value_mapping.get('ServiceUnderStoop',{})
AccessibleCustomer_map = value_mapping.get('AccessibleCustomer',{})
NoAccessReasonCdCustomer_map = value_mapping.get('NoAccessReasonCdCustomer',{})
CorrosionSeverityCdCustomer_map = value_mapping.get('CorrosionSeverityCdCustomer',{})
CorrosionLocationsCdCustomer_map = value_mapping.get('CorrosionLocationsCdCustomer',{})
CtagEquipmentCd_map = value_mapping.get('CtagEquipmentCd',{})
HeadOfServiceValveNotAccessibleHeight_map = value_mapping.get('HeadOfServiceValveNotAccessibleHeight',{})
GasTurnedOff_map = value_mapping.get('GasTurnedOff',{})
AllServicePipingInspected_map = value_mapping.get('AllServicePipingInspected',{})
CorrosionFoundOnCdCustomer_map = value_mapping.get('CorrosionFoundOnCdCustomer',{})
ImproperConditionsFoundCustomer_map = value_mapping.get('ImproperConditionsFoundCustomer',{})
DamagedGasMeterCustomerCd_map = value_mapping.get('DamagedGasMeterCustomerCd',{})
InappropriateFlexibleConnectorCds_map = value_mapping.get('InappropriateFlexibleConnectorCds',{})
AccessibleConEd_map = value_mapping.get('AccessibleConEd',{})
NoAccessReasonCdConed_map = value_mapping.get('NoAccessReasonCdConed',{})
ImproperSubstandardConditionCustomerCds_map = value_mapping.get('ImproperSubstandardConditionCustomerCds',{})
POESleeveLocationCd_map = value_mapping.get('POESleeveLocationCd',{})
InspDoneOnAllSvcsEnteringBldg_map = value_mapping.get('InspDoneOnAllSvcsEnteringBldg',{})
ImproperConditionsFoundConEd_map = value_mapping.get('ImproperConditionsFoundConEd',{})
ImproperSubstandardConditionConEdCds_map = value_mapping.get('ImproperSubstandardConditionConEdCds',{})
DamagedGasMeterConEdCd_map = value_mapping.get('DamagedGasMeterConEdCd',{})
UnauthorizedMetersFound_map = value_mapping.get('UnauthorizedMetersFound',{})
SupervisorNotified_map = value_mapping.get('SupervisorNotified',{})
PlasticExposed_map = value_mapping.get('PlasticExposed',{})
PlasticExtendsIntoStructure24Inches_map = value_mapping.get('PlasticExtendsIntoStructure24Inches',{})
CorrosionSeverityCdConEd_map = value_mapping.get('CorrosionSeverityCdConEd',{})
CorrosionFoundOnCdConEd_map = value_mapping.get('CorrosionFoundOnCdConEd',{})
CorrosionLocationsCdConEd_map = value_mapping.get('CorrosionLocationsCdConEd',{})
NGDRemovedReasonCd_map = value_mapping.get('NGDRemovedReasonCd',{})
PoeAccessible_map = value_mapping.get('PoeAccessible',{})
PoeNoAccessReasonCd_map = value_mapping.get('PoeNoAccessReasonCd',{})
PoeFoundSealedCondition_map = value_mapping.get('PoeFoundSealedCondition',{})
PoeLeftSealedCondition_map = value_mapping.get('PoeLeftSealedCondition',{})
PoeSealTypeCd_map = value_mapping.get('PoeSealTypeCd',{})
GasOdorPresent_map = value_mapping.get('GasOdorPresent',{})
GasLeakCorrosionRelated_map = value_mapping.get('GasLeakCorrosionRelated',{})
LeakItemCd_map = value_mapping.get('LeakItemCd',{})

In [ ]:
# file_to_read = input("WHICH DATABASE ARE YOU RUNNING? \n ---> RECONN \n ---> SLI \n ---> STORTI \n")

if os.path.basename(file_to_read) == 'RECONN':
    new_df['OutsideSet'] = new_df['OutsideSet'].map(OutsideSet_map).fillna('NO')
    # new_df['PressureRegulatorInBuilding'] = new_df['PressureRegulatorInBuilding'].map(PressureRegulatorInBuilding_map).fillna('YES')
    new_df['TypeOfDwellingCd'] = new_df['TypeOfDwellingCd'].map(TypeOfDwellingCd_map).fillna('SINGLE')
    new_df['MeterLocation'] = new_df['MeterLocation'].map(MeterLocation_map).fillna('METERROOM')
    new_df['MeterLocationAtmosphere'] = new_df['MeterLocationAtmosphere'].map(MeterLocationAtmosphere_map).fillna('METERRROOM')
    new_df['PipingConstructionMaterialCd'] = new_df['PipingConstructionMaterialCd'].map(PipingConstructionMaterialCd_map).fillna('METALLICBARE')
    new_df['MeterConstructionMaterialCd'] = new_df['MeterConstructionMaterialCd'].map(MeterConstructionMaterialCd_map).fillna('METALLICBARE')
    new_df['FittingsConstructionMaterialCd'] = new_df['FittingsConstructionMaterialCd'].map(FittingsConstructionMaterialCd_map).fillna('METALLICBARE')
    new_df['MainAndBranchService'] = new_df['MainAndBranchService'].map(MainAndBranchService_map).fillna('YES')
    new_df['NoAccessReasonCdBuilding'] = new_df['NoAccessReasonCdBuilding'].map(NoAccessReasonCdBuilding_map).fillna('OTHER')
    new_df['ServiceUnderStoop'] = new_df['ServiceUnderStoop'].map(ServiceUnderStoop_map).fillna('YES')
    new_df['AccessibleCustomer'] = new_df['AccessibleCustomer'].map(AccessibleCustomer_map).fillna('YES')
    new_df['NoAccessReasonCdCustomer'] = new_df['NoAccessReasonCdCustomer'].map(NoAccessReasonCdCustomer_map).fillna('OTHER')
    new_df['CorrosionSeverityCdCustomer'] = new_df['CorrosionSeverityCdCustomer'].map(CorrosionSeverityCdCustomer_map).fillna('NONE')
    new_df['CorrosionLocationsCdCustomer'] = new_df['CorrosionLocationsCdCustomer'].map(CorrosionLocationsCdCustomer_map).fillna('OTHER')
    new_df['CtagEquipmentCd'] = new_df['CtagEquipmentCd'].map(CtagEquipmentCd_map).fillna('')
    # new_df['HeadOfServiceValveNotAccessibleHeight'] = new_df['HeadOfServiceValveNotAccessibleHeight'].map(HeadOfServiceValveNotAccessibleHeight_map).fillna('OTHER')
    # new_df['GasTurnedOff'] = new_df['GasTurnedOff'].map(GasTurnedOff_map).fillna('NO')
    #fixed first_df['gas_turn_off'] to new_df['GasTurnedOff']
    new_df['GasTurnedOff'] = new_df['GasTurnedOff'].map(GasTurnedOff_map).fillna('NO')
    new_df['AllServicePipingInspected'] = new_df['AllServicePipingInspected'].map(AllServicePipingInspected_map).fillna('YES')
    new_df['CorrosionFoundOnCdCustomer'] = new_df['CorrosionFoundOnCdCustomer'].map(CorrosionFoundOnCdCustomer_map).fillna('OTHER')
    # new_df['ImproperConditionsFoundCustomer'] = new_df['ImproperConditionsFoundCustomer'].map(ImproperConditionsFoundCustomer_map).fillna('NO')
    # new_df['DamagedGasMeterCustomerCd'] = new_df['DamagedGasMeterCustomerCd'].map(DamagedGasMeterCustomerCd_map).fillna('OTHER')
    # new_df['InappropriateFlexibleConnectorCds'] = new_df['InappropriateFlexibleConnectorCds'].map(InappropriateFlexibleConnectorCds_map).fillna('')
    new_df['AccessibleConEd'] = new_df['AccessibleConEd'].map(AccessibleConEd_map).fillna('YES')
    new_df['NoAccessReasonCdConed'] = new_df['NoAccessReasonCdConed'].map(NoAccessReasonCdConed_map).fillna('DIDNOTATTEMPT')
    new_df['POESleeveLocationCd'] = new_df['POESleeveLocationCd'].map(POESleeveLocationCd_map).fillna('OTHER')
    new_df['InspDoneOnAllSvcsEnteringBldg'] = new_df['InspDoneOnAllSvcsEnteringBldg'].map(InspDoneOnAllSvcsEnteringBldg_map).fillna('YES')
    new_df['ImproperConditionsFoundConEd'] = new_df['ImproperConditionsFoundConEd'].map(ImproperConditionsFoundConEd_map).fillna('NO')
    new_df['ImproperSubstandardConditionConEdCds'] = new_df['ImproperSubstandardConditionConEdCds'].map(ImproperSubstandardConditionConEdCds_map).fillna('OTHER')
    # new_df['DamagedGasMeterConEdCd'] = new_df['DamagedGasMeterConEdCd'].map(DamagedGasMeterConEdCd_map).fillna('OTHER')
    new_df['UnauthorizedMetersFound'] = new_df['UnauthorizedMetersFound'].map(UnauthorizedMetersFound_map).fillna('NO')
    new_df['SupervisorNotified'] = new_df['SupervisorNotified'].map(SupervisorNotified_map).fillna('NO')
    new_df['PlasticExposed'] = new_df['PlasticExposed'].map(PlasticExposed_map).fillna('NO')
    new_df['PlasticExtendsIntoStructure24Inches'] = new_df['PlasticExtendsIntoStructure24Inches'].map(PlasticExtendsIntoStructure24Inches_map).fillna('YES')
    new_df['CorrosionSeverityCdConEd'] = new_df['CorrosionSeverityCdConEd'].map(CorrosionSeverityCdConEd_map).fillna('OTHER')
    new_df['CorrosionFoundOnCdConEd'] = new_df['CorrosionFoundOnCdConEd'].map(CorrosionFoundOnCdConEd_map).fillna('OTHER')
    new_df['CorrosionLocationsCdConEd'] = new_df['CorrosionLocationsCdConEd'].map(CorrosionLocationsCdConEd_map).fillna('OTHER')
    # new_df['NGDRemovedReasonCd'] = new_df['NGDRemovedReasonCd'].map(NGDRemovedReasonCd_map).fillna('OTHER')
    new_df['ImproperSubstandardConditionCustomerCds'] = new_df['ImproperSubstandardConditionCustomerCds'].map(ImproperSubstandardConditionCustomerCds_map).fillna('YES')
    new_df['PoeAccessible'] = new_df['PoeAccessible'].map(PoeAccessible_map).fillna('YES')
    new_df['PoeNoAccessReasonCd'] = new_df['PoeNoAccessReasonCd'].map(PoeNoAccessReasonCd_map).fillna('OTHER')
    new_df['PoeFoundSealedCondition'] = new_df['PoeFoundSealedCondition'].map(PoeFoundSealedCondition_map).fillna('NO')
    new_df['PoeLeftSealedCondition'] = new_df['PoeLeftSealedCondition'].map(PoeLeftSealedCondition_map).fillna('NO')
    # new_df['PoeSealTypeCd'] = new_df['PoeLeftSealedCondition'].map(PoeLeftSealedCondition_map).fillna('OTHER')
    new_df['PoeSealTypeCd'] = new_df['PoeSealTypeCd'].fillna('OTHER')
    new_df['GasOdorPresent'] = new_df['GasOdorPresent'].map(GasOdorPresent_map).fillna('NO')
    new_df['GasLeakCorrosionRelated'] = new_df['GasLeakCorrosionRelated'].map(GasLeakCorrosionRelated_map).fillna('NO')
    new_df['LeakItemCd'] = new_df['LeakItemCd'].map(LeakItemCd_map).fillna('OTHER')
    print("RUNNING RECONN")
    new_df = new_df[first_df['outside_set']!='YES']
    new_df = new_df[first_df['accessible']!='NO']

    new_df['Borough'] = first_df['borough']
    new_df['CompletedBy'] = first_df['_assigned_to']
    new_df['StartedDateTime']=first_df['_server_cr']
    new_df['AssignedDateTime']=first_df['_server_updated_at']
    new_df['NoOfMeters']=first_df['number_of_meters']
    new_df['NoOfApartments']=first_df['number_of_apts']
    new_df['AssetRemarks']=first_df['remarks']
    new_df['GasLeakReadingCGI']=first_df['cgi_reading_gas']
    new_df['GasLeakTicketNbr']=first_df['leak_ticket_']
    new_df['LeakItemRemarks']=first_df['remarks5']
    new_df['HouseNumber'] = first_df['address_sub_thoroughfare']
    new_df['StreetName'] = first_df['address_thoroughfare']
    new_df['ZipCode'] = first_df['address_postal_code'] 
    
    new_df['PressureRegulatorInBuilding'] = new_df['PressureRegulatorInBuilding'].apply(lambda x:'YES' if x==0 or x== 'yes' else 'NO')
    new_df['SupervisorNotified'] = new_df['SupervisorNotified'].apply(lambda x: 'YES' if x == 'yes' or x =='0' else 'NO')
    new_df['InspDoneOnAllSvcsEnteringBldg'] = new_df['InspDoneOnAllSvcsEnteringBldg'].apply(lambda x:'YES' if x == 0 or x == 'yes' else 'NO')
    new_df['ServiceUnderStoop'] = new_df['ServiceUnderStoop'].apply(lambda x: 'YES' if x == 0 or x == 'yes' else 'NO')
    new_df['MeterConstructionMaterialCd'] = new_df['MeterConstructionMaterialCd'].apply(lambda x: 'METALLICBARE' if x in ['METALLICBARE', 'METALLIC BARE', 'Steel-Bare', 'Cast Iron-Bare'] else 'METALLICCOAT' if x in ['METALLICCOAT', 'METALLIC COATED/PAINTED', 'Cast Iron-Coated/Painted', 'Steel-Coated/Painted'] else '')
    new_df['PipingConstructionMaterialCd'] = new_df['PipingConstructionMaterialCd'].apply(lambda x: 'METALLICCOAT' if x in ['METALLICCOAT', 'Steel-Coated/Painted'] else 'METALLICBARE' if x in [0, 'Copper', 'Steel-Bare'] else '')
    new_df['FittingsConstructionMaterialCd'] = new_df['FittingsConstructionMaterialCd'].apply(lambda x: 'METALLICBARE' if x in [0, 'METALLICBARE', 'Steel-Bare', 'Aluminum-Bare'] else 'METALLICCOAT' if x in ['METALLICCOAT', 'Steel-Coated/Painted', 'Aluminum-Coated/Painted'] else '')
    new_df['UnauthorizedMetersFound'] = new_df['UnauthorizedMetersFound'].apply(lambda x: 'YES' if x == 'yes' else 'NO' if x in [0, 'no'] else '')
    new_df['ImproperSubstandardConditionCustomerCds'] = new_df['ImproperSubstandardConditionCustomerCds'].apply(lambda x: 'YES' if x == 0 else x)
    new_df['TypeOfDwellingCd'] = new_df.apply(lambda row: '' if row['TypeOfDwellingCd'] == 0 else 'MULTIUNIT' if row['NoOfMeters'] > 2 else TypeOfDwellingCd_map.get(row['TypeOfDwellingCd'],'OTHER') if row['TypeOfDwellingCd'] and pd.notna(row['TypeOfDwellingCd']) else 'SINGLE', axis=1)
    new_df['PoeNoAccessReasonCd'] = new_df.apply(lambda row: '' if row['PoeAccessible'] == 'YES' else row['PoeNoAccessReasonCd'], axis = 1)
    new_df['GasOdorPresent'] = new_df['GasOdorPresent'].apply(lambda x: 'YES' if x == 'yes' else 'NO')
    new_df['NoAccessReasonCdCustomer'] = new_df.apply(lambda row: '' if row['AccessibleCustomer'] in ['YES', 0] else row['NoAccessReasonCdCustomer'], axis=1)
    
    for column, mapping in value_mapping.items():
        if column != 'WR#' and column != 'InspectionType' and column != 'Muni' and column !='StatedDateTime' and column != 'AssignedDateTime' and column != 'NoOfMeters' and column != 'NoOfApartments' and column != 'AssetRemarks' and column != 'GasLeakReadingCGI' and column != 'GasLeakTicketNbr' and column != 'LeakItemRemarks' and column != 'ServiceSizeCd' and column != 'ServiceMaterialCd' and column != 'InspectionDateTime' and column != 'MeterLocationRemarks' and column != 'MeterLocationAtmosphereRemarks' and column != 'NoAccessReasonCustomerRemarks' and column != 'CorrosionSeverityCdCustomer' and column != 'GasTurnedOff' and column != 'CorrosionLocationCustomerRemarks' and column != 'ImproperConditionsFoundCustomer' and column != 'HeadOfServiceValveNotAccessibleHeight' and column != 'DamagedGasMeterCustomerCd' and column != 'INAPPRGROUNDPIPINGCD' and column != 'InappropriateFlexibleConnectorCds' and column != 'ImproperSubstandardConditionsCustomerOtherDescribe' and column != 'CorrosionInspectionRemarks' and column != 'DamagedGasMeterConEdCd':
            new_df[column] = new_df[column].replace(mapping).fillna(mapping.get(np.nan,np.nan))
    
    new_df.to_excel('RECONN_parsed.xlsx',index=False)
     
    
elif os.path.basename(file_to_read) == 'SLI':
    print("RUNNING SLI")
    
    new_df = new_df[first_df['All Meters Outdoors for Services Entering Building ']!='YES']
    new_df = new_df[first_df['Is the building accessible?']!='NO']
    
    new_df['BuildingAccessible'] = new_df['BuildingAccessible'].fillna('YES')
    new_df['OutsideSet'] = new_df['OutsideSet'].fillna('NO')
    new_df['PressureRegulatorInBuilding'] = new_df['PressureRegulatorInBuilding'].fillna('YES')
    new_df['TypeOfDwellingCd'] = new_df['TypeOfDwellingCd'].fillna('SINGLE')
    new_df['MeterLocation'] = new_df['MeterLocation'].fillna('BASEMENT')
    new_df['MeterLocationAtmosphere'] = new_df['MeterLocationAtmosphere'].fillna('METERROOM')
    new_df['PipingConstructionMaterialCd'] = new_df['PipingConstructionMaterialCd'].fillna('METALLICBARE')
    new_df['MeterConstructionMaterialCd'] = new_df['MeterConstructionMaterialCd'].fillna('METALLICBARE')
    new_df['FittingsConstructionMaterialCd'] = new_df['FittingsConstructionMaterialCd'].fillna('METALLICBARE')
    new_df['MainAndBranchService'] = new_df['MainAndBranchService'].fillna('YES')
    new_df['NoAccessReasonCdBuilding'] = new_df['NoAccessReasonCdBuilding'].fillna('OTHER')
    new_df['ServiceUnderStoop'] = new_df['ServiceUnderStoop'].fillna('YES')
    new_df['AccessibleCustomer'] = new_df['AccessibleCustomer'].fillna('YES')
    new_df['NoAccessReasonCdCustomer'] = new_df['NoAccessReasonCdCustomer'].fillna('OTHER')
    new_df['CorrosionSeverityCdCustomer'] = new_df['CorrosionSeverityCdCustomer'].fillna('NONE')
    new_df['CorrosionLocationsCdCustomer'] = new_df['CorrosionLocationsCdCustomer'].fillna('NONE')
    new_df['CtagEquipmentCd'] = new_df['CtagEquipmentCd'].fillna('OES')
    new_df['HeadOfServiceValveNotAccessibleHeight'] = new_df['HeadOfServiceValveNotAccessibleHeight'].fillna('NO')
    new_df['GasTurnedOff'] = new_df['GasTurnedOff'].fillna('NO')
    new_df['AllServicePipingInspected'] = new_df['AllServicePipingInspected'].fillna('YES')
    new_df['CorrosionFoundOnCdCustomer'] = new_df['CorrosionFoundOnCdCustomer'].fillna('OTHER')
    new_df['ImproperConditionsFoundCustomer'] = new_df['ImproperConditionsFoundCustomer'].fillna('NO')
    new_df['DamagedGasMeterCustomerCd'] = new_df['DamagedGasMeterCustomerCd'].fillna('OTHER')
    new_df['InappropriateFlexibleConnectorCds'] = new_df['InappropriateFlexibleConnectorCds'].fillna('')
    new_df['AccessibleConEd'] = new_df['AccessibleConEd'].fillna('YES')
    new_df['NoAccessReasonCdConed'] = new_df['NoAccessReasonCdConed'].fillna('DIDNOTATTEMPT')
    new_df['POESleeveLocationCd'] = new_df['POESleeveLocationCd'].fillna('OTHER')
    new_df['InspDoneOnAllSvcsEnteringBldg'] = new_df['InspDoneOnAllSvcsEnteringBldg'].fillna('YES')
    new_df['ImproperConditionsFoundConEd'] = new_df['ImproperConditionsFoundConEd'].fillna('NO')
    new_df['ImproperSubstandardConditionConEdCds'] = new_df['ImproperSubstandardConditionConEdCds'].fillna('OTHER')
    new_df['DamagedGasMeterConEdCd'] = new_df['DamagedGasMeterConEdCd'].fillna('OTHER')
    new_df['UnauthorizedMetersFound'] = new_df['UnauthorizedMetersFound'].fillna('NO')
    new_df['SupervisorNotified'] = new_df['SupervisorNotified'].fillna('YES')
    new_df['PlasticExposed'] = new_df['PlasticExposed'].fillna('NO')
    new_df['PlasticExtendsIntoStructure24Inches'] = new_df['PlasticExtendsIntoStructure24Inches'].fillna('YES')
    new_df['CorrosionSeverityCdConEd'] = new_df['CorrosionSeverityCdConEd'].fillna('NONE')
    new_df['CorrosionFoundOnCdConEd'] = new_df['CorrosionFoundOnCdConEd'].fillna('OTHER')
    new_df['CorrosionLocationsCdConEd'] = new_df['CorrosionLocationsCdConEd'].fillna('OTHER')
    new_df['NGDRemovedReasonCd'] = new_df['NGDRemovedReasonCd'].fillna('OTHER')
    new_df['PoeAccessible'] = new_df['PoeAccessible'].fillna('YES')
    new_df['PoeNoAccessReasonCd'] = new_df['PoeNoAccessReasonCd'].fillna('OTHER')
    new_df['PoeFoundSealedCondition'] = new_df['PoeFoundSealedCondition'].fillna('NO')
    new_df['PoeLeftSealedCondition'] = new_df['PoeLeftSealedCondition'].fillna('NO')
    new_df['PoeSealTypeCd'] = new_df['PoeSealTypeCd'].fillna('OTHER')
    new_df['GasOdorPresent'] = new_df['GasOdorPresent'].fillna('NO')
    new_df['GasLeakCorrosionRelated'] = new_df['GasLeakCorrosionRelated'].fillna('NO')
    new_df['LeakItemCd'] = new_df['LeakItemCd'].fillna('OTHER')

    new_df['InspectionDateTime'] = first_df['Date of the Inspection']
    new_df['StartedDateTime'] = first_df['Start time']
    new_df['HouseNumber'] = first_df['House Number']
    new_df['CompassPoint'] = first_df['Compass Point']
    new_df['StreetName'] = first_df['Street Name']
    new_df['Artery'] = first_df['Artery']
    new_df['ZipCode'] = first_df['Zipcode']
    new_df['Borough'] = first_df['Borough']
    new_df['Muni'] = first_df['Municipality']
    new_df['TypeOfDwellingRemarks'] = first_df['Number of Apartments']
    new_df['NoOfMeters'] = first_df['Number of Meters']
    new_df['NoOfApartments'] = first_df['Number of Apartments']
    new_df['AssetRemarks'] = first_df['Asset Remarks']
    new_df['CtagEquipmentRemarks'] = first_df['Tag equipment remarks']
    new_df['CorrosionInspectionRemarks'] = first_df['Remarks']
    new_df['RemarksConEd'] = first_df['General Remarks']
    new_df['PoeRemarks'] = first_df['Gas POE Remarks']
    new_df['GasLeakReadingCGI'] = first_df['C.G.I. Reading (Percent Gas)']
    new_df['GasLeakTicketNbr'] = first_df['Leak ticket number']
    new_df['LeakInspectionRemarks'] = first_df['Leak Inspection Remarks']
    new_df['Employee_No'] = first_df['Please Enter Inspector Employee ID']

    new_df['TypeOfDwellingCd'] = new_df['TypeOfDwellingCd'].apply(lambda x: TypeOfDwellingCd_map.get(str(x).lower(), TypeOfDwellingCd_map.get(x, 'OTHER')) if x and pd.notna(x) else 'SINGLE')
    new_df['CorrosionSeverityCdCustomer'] = new_df['CorrosionSeverityCdCustomer'].astype(str).apply(lambda x: CorrosionSeverityCdCustomer_map.get(x.lower(), 'NONE'))
    # new_df['CorrosionSeverityCdCustomer'] = new_df['CorrosionSeverityCdCustomer'].apply(lambda x: CorrosionSeverityCdCustomer_map.get(x.lower(), 'NONE'))
    new_df['CtagEquipmentCd'] = new_df.apply(lambda row: CtagEquipmentCd_map.get(row['CorrosionSeverityCdCustomer'], row['CtagEquipmentCd']), axis=1)
    new_df['PipingConstructionMaterialCd'] = new_df['PipingConstructionMaterialCd'].apply(lambda x: PipingConstructionMaterialCd_map.get(x, 'OTHER') if x and pd.notna(x) else 'METALLICBARE')
    new_df['MeterConstructionMaterialCd'] = new_df['MeterConstructionMaterialCd'].apply(lambda x: MeterConstructionMaterialCd_map.get(x, 'OTHER') if x and pd.notna(x) else 'METALLICBARE')
    new_df['FittingsConstructionMaterialCd'] = new_df['FittingsConstructionMaterialCd'].apply(lambda x: FittingsConstructionMaterialCd_map.get(x, 'OTHER') if x and pd.notna(x) else 'METALLICBARE')
    new_df['MainAndBranchService'] = new_df['MainAndBranchService'].apply(lambda x: MainAndBranchService_map.get(x, 'OTHER') if x and pd.notna(x) else 'YES')
    new_df['NoOfMeters'] = new_df.apply(lambda row: 1 if row['TypeOfDwellingCd'] == 'SINGLE' else row['NoOfMeters'], axis=1)
    new_df['NoOfApartments'] = new_df.apply(lambda row: '' if row['TypeOfDwellingCd'] == 'SINGLE' else row['NoOfApartments'], axis=1)

    for column, mapping in value_mapping.items():
        if column!='TypeOfDwellingCd' and column!='MeterLocation' and column!='PipingConstructionMaterialCd' and column!='MeterConstructionMaterialCd' and column!='MainAndBranchService' and column!='FittingsConstructionMaterialCd':
            new_df[column] = new_df[column].replace(mapping)
            
    new_df.to_excel('SLI_parsed.xlsx',index=False)
    
elif os.path.basename(file_to_read) == 'STORTI':
    print("Not completed yet")
    new_df.to_excel('STORTI_parsed.xlsx',index=False)
      
else:
    raise ValueError("Your input is incorrect, please enter one of the three again.")

In [ ]:
df = address_df.rename(columns={'HouseNumber': 'raw address'})
addresses = df['raw address'].tolist()

def process_data(address):
    if not isinstance(address, str):
      return ['', '', '', '', '']

    house_number_re = re.compile(r'^[\d-]+')
    compass_point_re = re.compile(r'(?<= )([NESW]|North|East|South|West)(?= )', re.IGNORECASE)
    street_name_re = re.compile(r'(?:(?<=[NESW] )|(?<= ))[\w\s-]+(?= (St|Street|Rd|Road|Ave|Avenue|Blvd|Boulevard))', re.IGNORECASE)
    artery_re = re.compile(r'(?<= )(St|Street|Rd|Road|Ave|Avenue|Blvd|Boulevard)', re.IGNORECASE)
    zip_code_re = re.compile(r'\b\d{5}\b')

    compass_dict = {
    's': 'South',
    'n': 'North',
    'e': 'East',
    'w': 'West',
    }

    artery_dict = {
    'st': 'Street',
    'av': 'Avenue',
    'ave': 'Avenue',
    'rd': 'Road',
    'blvd':'Boulevard'
    }

    address = re.sub(r'United States', '', address, flags=re.IGNORECASE)
    address = re.sub(r'New York', '', address, 1, flags=re.IGNORECASE) if address.lower().count('new york') > 1 else address

    house_number = house_number_re.search(address)
    house_number = house_number.group() if house_number else ''

    compass_point = compass_point_re.search(address)
    if compass_point:

      compass_point = compass_point.group()
      oc = compass_point
      compass_point = compass_dict.get(compass_point.lower(),compass_point)
    else:
      compass_point= ''

    street_name = street_name_re.search(address)
    if street_name:
        street_name = street_name.group().strip()
        # If the street name starts with a compass point, remove it
        if compass_point and street_name.startswith(oc):
            street_name = street_name[len(oc)+1:].strip() # +1 added to handle the space after compass point.
    else:
        street_name = ''

    artery = artery_re.search(address)
    if artery:
        artery = artery.group()
        artery = artery_dict.get(artery.lower(), artery)
    else:
        artery = ''

    zip_code = zip_code_re.search(address)
    zip_code = zip_code.group() if zip_code else ''

    return [house_number, compass_point, street_name, artery, zip_code]


formatted_addresses = [process_data(address) for address in addresses]
# print(formatted_addresses)

df_new = pd.DataFrame(formatted_addresses, columns=['HouseNumber', 'CompassPoint', 'StreetName', 'Artery', 'ZipCode'])

# concatenate the original DataFrame with the new one
# df_final = pd.concat([df, df_new], axis=1)

# df_new.to_excel('RECONN_ADRESS_PARSED.xlsx', index=False)

# Load the workbook
# book = load_workbook('reconn_parsed.xlsx')

# Create a pandas Excel writer object
with pd.ExcelWriter('reconn_parsed.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:            
    df_new.to_excel(writer, 'Sheet1', index=False, header=True,startcol=10)

# Write df_final to the Excel file starting at column K
# df_new.to_excel("RECONN_ADDRESS_PARSED", index=False)

In [ ]:
# Initialize an empty list to store processed data
processed_data = []

# Iterate through each row of the address DataFrame
for index, row in address_df.iterrows():
    address = str(row['raw address'])
    
    # Check if the address should be skipped
    if pd.isna(address) or "SKIP" in str(address):
        continue  # Skip this address and move to the next iteration

    # Extract house number, compass, street name, and artery name
    #house_number_match = re.search(r'^[\d-]+', address)
    house_number_match = re.search(r'^[A-Z\d-]+', address)
    house_number = house_number_match.group() if house_number_match is not None else ''

    # Extract compass point if present in the address
    compass_point_match = re.search(r'(?<!\w)([NESW]|North|East|South|West)(?!\w)', address, re.IGNORECASE)
    compass_point = compass_point_match.group() if compass_point_match else ''

    # Split the address into words
    address_words = address.split()

    # Extract the last word as the artery name
    artery_name = address_words[-1] if address_words else ''

    # Remove the last word from the address_words list
    address_words = address_words[:-1]

    # # Extract street name from the remaining words
    # street_name = ' '.join(address_words)

    # Extract street name from the remaining words without house number and compass
    street_name = ' '.join([word for word in address_words if word not in [house_number, compass_point]])

    # Include first letter of street name if it's the same as compass
    if compass_point and street_name.lower().startswith(compass_point.lower()):
        street_name = street_name[len(compass_point):].strip()

    # If compass is present, try to capture the street name after compass and before artery
    if compass_point and not street_name:
        street_name_match = re.search(rf'{compass_point}\s*(\w+)\s+[A-Z]+', address, re.IGNORECASE)
        street_name = street_name_match.group(1) if street_name_match else ''

    # Find replacement for street name using mapping
    street_mapping = street_mapping_df.set_index('StreetName')['Standard Notation'].to_dict()
    if street_name.upper() in street_mapping:
        street_name = street_mapping[street_name.upper()]

    artery_mapping = artery_mapping_df.set_index('StreetNamePostType')['Standard Notation'].to_dict()
    if artery_name.upper() in artery_mapping:
        artery_name = artery_mapping[artery_name.upper()]

    # Append the processed data to the list
    processed_data.append([house_number, compass_point, street_name, artery_name, address])

# Create a new DataFrame with processed data
processed_df = pd.DataFrame(processed_data, columns=['House Number', 'Compass', 'Street Name', 'Artery Name', 'ADDRESS'])

# Write the processed DataFrame to a new Excel file
processed_df.to_excel('RECONN_ADDRESS_PARSED.xlsx', index=False)

In [ ]:
print(first_df['pressure_regulator_in_building'].unique())  # Print unique values in new_df['GasTurnedOff']
print(OutsideSet_map.keys())  # Print keys in GasTurnedOff_map
print(new_df['OutsideSet'].isna().sum())  # Print number of NaN values in new_df['GasTurnedOff']
print(first_df['pressure_regulator_in_building'].unique())

In [ ]:
print(new_df)